<a href="https://colab.research.google.com/github/raj-vijay/da/blob/master/10_Backtracking_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Backtracking Search**

**BACKGROUND**

This exercise will build on Lab07 and Lab08 and add a backtracking search algorithm for finding all feasible solutions to a SAT problem. 

**Task 1**

<p align = 'justify'>Use the model class from Lab07 or Lab08 and add a function Search() to implement a backtracking search with integrated arc consistency checking to find all valid solutions to the SAT problem. Add and remove constraints to the model and make sure to enforce arc consistency in each node.</p>

<p align ='justify'>(Hint: use copy.deepcopy to create copies of the model when branching and backtracking; now you need to make sure that variables are not compared by references but using their names in the constraint propagation, as the deep copy will make a copy of the variables)</p>


**Task 2**

<p align = 'justify'>Create a model for the N-queens problem and solve the problem using the SAT solver implemented in task 1.</p>

In [ ]:
!pip install ortools

     |████████████████████████████████| 14.0MB 263kB/s 
     |████████████████████████████████| 1.0MB 45.8MB/s 
     |████████████████████████████████| 133kB 46.8MB/s 
ERROR: tensorflow-metadata 0.25.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0


In [ ]:
import copy
import itertools

In [ ]:
class BoolVariable():
    def __init__(self,name):
        self.name_ = name
        self.domain_ = [True,True]

    def RemoveFromDomain(self,v):
        if v:
            self.domain_[1]=False
        else:
            self.domain_[0]=False            
    
    def GetName(self):
        return self.name_
    
    def IsInList(self, list):
        for l in list:
            if self.name_ == l.name_:
                return True
        return False
    
    def GetValues(self):
        values = []
        if self.domain_[0]:
            values.append(False)
        if self.domain_[1]:
            values.append(True)
        return values

In [ ]:
class Constraint():
    def CheckConstraint(self, x,v):
        result = False            
        for t in self.combinations_:                                
            is_valid = True
            for s in self.scheme_:
                values = s.GetValues()
                if len(values)==1:
                    if not ((values[0] and s.IsInList(t)) or ((not values[0]) and (not s.IsInList(t)))):
                        is_valid = False
            if is_valid:
                if (v and (x.IsInList(t)) or ((not v) and (not x.IsInList(t)))):                                        
                    result = True
                    break
        return result        

    def Print(self):
        for t in self.combinations_:
            for x in self.scheme_:
                if (x.IsInList(t)):
                    print(x.name_, end=",")
                else:
                    print("not",x.name_, end=",")
            print()
        print()

In [ ]:
class OrConstraint(Constraint):
    def __init__(self, positive, negative):
        self.scheme_ = []
        for p in positive:
            self.scheme_.append(p)
        for n in negative:
            self.scheme_.append(n)
        self.combinations_ = self.GetAllCombinations_(positive,negative)
#        self.Print()

    def GetAllCombinations_(self,positive,negative):        
        result = []
        for length in range(len(self.scheme_)+1):
            for subset in itertools.combinations(self.scheme_,length):
                valid = False
                for x in self.scheme_:
                    if x.IsInList(subset):
                        if x.IsInList(positive):
                            valid = True
                            break
                    else: 
                        if x.IsInList(negative):
                            valid = True
                            break
                if valid:
                    result.append(subset)
        return result

In [ ]:
class AndConstraint(Constraint):
    def __init__(self, positive, negative, onlyenforceif, onlyenforceifnot):
        self.scheme_ = []
        for p in positive:
            self.scheme_.append(p)
        for n in negative:
            self.scheme_.append(n)
        for p in onlyenforceif:
            self.scheme_.append(p)
        for n in onlyenforceifnot:
            self.scheme_.append(n)
        self.combinations_ = self.GetAllCombinations_(positive,negative,onlyenforceif,onlyenforceifnot)
        # self.Print()

    def GetAllCombinations_(self,positive,negative,onlyenforceif,onlyenforceifnot):        
        result = []
        for length in range(len(self.scheme_)+1):
            for subset in itertools.combinations(self.scheme_,length):                
                enforce = True
                for x in onlyenforceif:
                    if not x.IsInList(subset):
                        enforce = False
                for x in onlyenforceifnot:
                    if x.IsInList(subset):
                        enforce = False            
                valid = True
                for x in positive:
                    if not x.IsInList(subset):
                        valid = False
                for x in negative:
                    if x.IsInList(subset):
                        valid = False
                if not enforce or valid:
                    result.append(subset)
        return result

In [ ]:
class Model():
    def __init__(self):
        self.variables_ = []
        self.constraints_ = []
        
    def AddBoolVariable(self, name):
        v = BoolVariable(name)
        self.variables_.append(v)
        return v
        
    def AddOrConstraint(self, positive, negative):
        c = OrConstraint(positive,negative)
        self.constraints_.append(c)

    def AddAndConstraint(self, positive, negative, onlyenforceif,onlyenforceifnot):
        c = AndConstraint(positive,negative,onlyenforceif,onlyenforceifnot)
        self.constraints_.append(c)

    def PrintDomains(self):
        for x in self.variables_:
            print(x.GetName(), x.GetValues())
        print()

    def Revise_(self,x,c):
        change = False
        for v in x.GetValues():
            if not c.CheckConstraint(x,v):
                x.RemoveFromDomain(v)
                change = True
        return change

    def AC3(self):
        Q = []
        for x in self.variables_:
            for c in self.constraints_:
                if x.IsInList(c.scheme_):
                    Q.append((x,c))
        while len(Q)>0:
            (x,c) = Q[0]
            Q.remove(Q[0])
            if (self.Revise_(x,c)):
                if len(x.GetValues())==0:
                    return False
                else:
                    for c2 in self.constraints_:
                        if (c2 != c) and (x.IsInList(c2.scheme_)):
                            for x2 in c2.scheme_:
                                if not (x2,c2) in Q:
                                    Q.append((x2,c2))
        return True
  
    def Search(self):
        if self.AC3():
            solution = True
            for x in self.variables_:
                if len(x.GetValues())>1:
                    solution = False
                    m1 = copy.deepcopy(self)
                    m1.AddOrConstraint([x],[])
                    m1.Search()
                    m2 = copy.deepcopy(self)
                    m2.AddOrConstraint([],[x])
                    m2.Search()
                    break
            if solution:
                self.PrintDomains()

In [ ]:
model = Model()

In [ ]:
N = 4

In [ ]:
board = {}

In [ ]:
for i in range(N):
        for j in range(N):
            board[(i,j)] = model.AddBoolVariable(str(i)+"_"+str(j))

In [ ]:
for i in range(N):
        for j in range(N):
            horizontal = [board[(i,k)] 
                          for k in range(N) if k!=j]
            vertical = [board[(k,j)] 
                        for k in range(N) if k!=i]
            diagonal1 = [board[(i+k,j+k)] 
                          for k in range(-N,N) 
                          if k!=0 and i+k>=0 and j+k>=0 and i+k<N and j+k<N]
            diagonal2 = [board[(i+k,j-k)] 
                          for k in range(-N,N) 
                          if k!=0 and i+k>=0 and j-k>=0 and i+k<N and j-k<N]
            threat = horizontal + vertical + diagonal1 + diagonal2
            model.AddAndConstraint([],threat,[board[(i,j)]],[])

In [ ]:
for i in range(N):
        horizontal = [board[(i,k)] 
                      for k in range(N)]
        vertical = [board[(k,j)] 
                    for k in range(N)]
        model.AddOrConstraint(horizontal,[])
        model.AddOrConstraint(vertical,[])

In [ ]:
model.Search()

0_0 [False]
0_1 [True]
0_2 [False]
0_3 [False]
1_0 [False]
1_1 [False]
1_2 [False]
1_3 [True]
2_0 [True]
2_1 [False]
2_2 [False]
2_3 [False]
3_0 [False]
3_1 [False]
3_2 [True]
3_3 [False]

0_0 [False]
0_1 [False]
0_2 [True]
0_3 [False]
1_0 [True]
1_1 [False]
1_2 [False]
1_3 [False]
2_0 [False]
2_1 [False]
2_2 [False]
2_3 [True]
3_0 [False]
3_1 [True]
3_2 [False]
3_3 [False]

